In [1]:
import os
from  dotenv import load_dotenv
load_dotenv()

True

In [2]:
if os.environ['OPEN_API_SECRET_KEY']:
    print("OpenAI API key is set")
else:
    print("OpenAI API key is not set")

OpenAI API key is set


## We will use local llm model ollama instead of openai paid, so any of you can test locally without spending bucks

#### Let's implement a RAG system using local LLMs via Ollama and interacting with a SQL database using LangChain.

In [20]:
## incase you are using openai api
# from langchain_openai import ChatOpenAI 
# llm = ChatOpenAI(model_name="gpt-<version>", temperature=0)

# from langchain_ollama import ChatOllama
## incase you are using ollama api
from langchain_community.chat_models import ChatOllama
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain


llm = ChatOllama(model="llama3", temperature=0)
database = SQLDatabase.from_uri("sqlite:///demo.db")
chain = SQLDatabaseChain(llm=llm, database=database, verbose=True, return_direct=True)

query = ("What is the total revenue of Headphones, aggregate on product name?")

result = chain.run(query)
print("Result: ", result)




> Entering new SQLDatabaseChain chain...
What is the total revenue of Headphones, aggregate on product name?
SQLQuery:Here's the answer:

Question: What is the total revenue of Headphones, aggregate on product_name?
SQLQuery:
SELECT "product_name", SUM("price") AS "Total Revenue"
FROM "Sales"
WHERE "product_name" = 'Headphones'
GROUP BY "product_name";
SQLResult: [('Headphones', 200.0)]
> Finished chain.
Result:  [('Headphones', 200.0)]


# Let's implement a RAG system using local LLMs via Ollama and interacting with a SQL database using LangChain.
#### Step 1: We will pass our text to the LLM and get the response. (Prepare Document for your text data)

In [59]:
from langchain_core.documents import Document

# Your text data
my_text = """ The team lead is Owais Ajaz in QoreLogix, now you guys will see the magic of local llm. before context the llm didnt 
but we have build the context now be happy"""

doc = Document(page_content=my_text, metadata={"source": "wikipedia", "version": 1})

print(doc)

page_content=' The team lead is Owais Ajaz in QoreLogix, now you guys will see the magic of local llm. before context the llm didnt 
but we have build the context now be happy' metadata={'source': 'wikipedia', 'version': 1}


#### Step 2: Splitting the document into chunks

In [60]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunks = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=25,
    length_function=len,
    is_separator_regex=False,
).split_documents([doc])

chunks

[Document(metadata={'source': 'wikipedia', 'version': 1}, page_content='The team lead is Owais Ajaz in QoreLogix, now you guys will see the magic of local llm. before context the llm didnt \nbut we have build the context now be happy')]

#### Step 3: Now its tom time to build a vector database, we will convert the chunks to embeddings and store them in a vector database.

In [55]:
from langchain_openai import OpenAIEmbeddings

## incase you are using openai embeddings
# embeddings_model = OpenAIEmbeddings(model="text-embedding-5-small")
# embeddings_model.embed_query("Hello world")

from langchain_community.embeddings import OllamaEmbeddings
embeddings_model = OllamaEmbeddings(model="llama3")
# embeddings_model.embed_query("Hello world")

In [62]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings_model
    # collection_name="my_collection",
    # persist_directory="./chroma_db"
)

context = vectorstore.similarity_search("who is team lead in QoreLogix?", k=2)

context

[Document(metadata={'version': 1, 'source': 'wikipedia'}, page_content='The team lead is M Owais Ajaz in qorelogix, now you guys will see the magic of local llm. before context the llm didnt \nbut we have build the context now be happy'),
 Document(metadata={'source': 'wikipedia', 'version': 1}, page_content='For example, this helps LLM-based chatbots access internal company data or generate responses based on authoritative sources.')]

In [ ]:
response = llm.invoke(f"who is team lead in QoreLogix? use the context provided: {context}")
print(response.content)

I apologize, but you didn't provide any context. Could you please provide more information about QoreLogix and what you mean by "Team Lead"? I'll do my best to help you with your question.
